In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers pandas sentence-transformers spacy

In [8]:
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load LLM response
llm_response_df = pd.read_csv('/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/With RAG/RAG-Llama.csv')

# Load reference texts
reference_files = {
    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",
    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",
    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",
}

reference_texts = {}
for source, file_path in reference_files.items():
    reference_texts[source] = pd.read_csv(file_path)

# Initialize a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return set([ent.text for ent in doc.ents])

# Function to compare LLM response with reference texts
def fact_check(llm_response, reference_texts):
    llm_embedding = model.encode(llm_response, convert_to_tensor=True)
    llm_entities = extract_entities(llm_response)
    similarities = {}
    missing_points = {}
    extra_points = {}

    for source, texts in reference_texts.items():
        max_similarity = 0
        reference_entities = set()
        for text in texts['cause']:
            ref_embedding = model.encode(text, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(llm_embedding, ref_embedding).item()
            if similarity > max_similarity:
                max_similarity = similarity
                reference_entities = extract_entities(text)
        similarities[source] = max_similarity
        missing_points[source] = reference_entities - llm_entities
        extra_points[source] = llm_entities - reference_entities

    return similarities, missing_points, extra_points

# Evaluate all rows in the LLM response DataFrame
results = []
for index, row in llm_response_df.iterrows():
    llm_response = row['Cause']
    similarities, missing_points, extra_points = fact_check(llm_response, reference_texts)
    results.append({
        'Index': index,
        'LLM Response': llm_response,
        'MayoClinic Similarity': similarities.get('MayoClinic', 0),
        'Healthline Similarity': similarities.get('Healthline', 0),
        'MedlinePlus Similarity': similarities.get('MedlinePlus', 0),
        'MayoClinic Missing Points': ', '.join(missing_points.get('MayoClinic', set())),
        'Healthline Missing Points': ', '.join(missing_points.get('Healthline', set())),
        'MedlinePlus Missing Points': ', '.join(missing_points.get('MedlinePlus', set())),
        'MayoClinic Extra Points': ', '.join(extra_points.get('MayoClinic', set())),
        'Healthline Extra Points': ', '.join(extra_points.get('Healthline', set())),
        'MedlinePlus Extra Points': ', '.join(extra_points.get('MedlinePlus', set()))
    })

# Convert results to a DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('RAG-Llama_fact_check.csv', index=False)

print("Results for all responses saved to Gemma_fact_check.csv")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Results for all responses saved to Gemma_fact_check.csv


# Factual accuracy

In [3]:
"""
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load LLM response
llm_response_df = pd.read_csv('/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/MixtralAI_advice.csv')

# Load reference texts
reference_files = {
    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",
    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",
    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",
}

reference_texts = {}
for source, file_path in reference_files.items():
    reference_texts[source] = pd.read_csv(file_path)

# Initialize a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return set([ent.text for ent in doc.ents])

# Function to compare LLM response with reference texts
def fact_check(llm_response, reference_texts):
    llm_embedding = model.encode(llm_response, convert_to_tensor=True)
    llm_entities = extract_entities(llm_response)
    similarities = {}
    entity_matches = {}
    missing_points = {}
    extra_points = {}

    for source, texts in reference_texts.items():
        max_similarity = 0
        matched_entities = set()
        for text in texts['cause']:
            ref_embedding = model.encode(text, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(llm_embedding, ref_embedding).item()
            if similarity > max_similarity:
                max_similarity = similarity
                matched_entities = extract_entities(text).intersection(llm_entities)
                missing_points[source] = extract_entities(text) - llm_entities
                extra_points[source] = llm_entities - extract_entities(text)
        similarities[source] = max_similarity
        entity_matches[source] = matched_entities

    return similarities, entity_matches, missing_points, extra_points

# Evaluate all rows in the LLM response DataFrame
results = []
for index, row in llm_response_df.iterrows():
    llm_response = row['Cause']
    similarities, entity_matches, missing_points, extra_points = fact_check(llm_response, reference_texts)
    results.append({
        'Index': index,
        'LLM Response': llm_response,
        'Similarities': similarities,
        'Entity Matches': entity_matches,
        'Missing Points': missing_points,
        'Extra Points': extra_points
    })

# Convert results to a DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('Gemma_fact_check.csv', index=False)

print("Results for all responses saved to fact_check_all_results.csv")
"""

'\nimport pandas as pd\nimport numpy as np\nfrom transformers import pipeline\nfrom sentence_transformers import SentenceTransformer, util\nimport spacy\n\n# Load spaCy model for NER\nnlp = spacy.load(\'en_core_web_sm\')\n\n# Load LLM response\nllm_response_df = pd.read_csv(\'/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/MixtralAI_advice.csv\')\n\n# Load reference texts\nreference_files = {\n    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",\n    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",\n    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",\n}\n\nreference_texts = {}\nfor source, file_path in reference_files.items():\n    reference_texts[source] = pd.read_csv(file_path)\n\n# Initialize a sentence transformer model for semantic similarity\nmodel = SentenceTransformer(\'all-MiniLM-L6-v2\')\n\n# Function to extract named entities\ndef extract_entities(text):\n    doc = nlp(text)\n

In [ ]:
"""
# Install necessary libraries
!pip install transformers pandas sentence-transformers spacy

import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load LLM response
llm_response_df = pd.read_csv('/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/Llama_advice.csv')

# Load reference texts
reference_files = {
    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",
    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",
    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",
}

reference_texts = {}
for source, file_path in reference_files.items():
    reference_texts[source] = pd.read_csv(file_path)

# Initialize a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return set([ent.text for ent in doc.ents])

# Function to compare LLM response with reference texts
def fact_check(llm_response, reference_texts):
    llm_embedding = model.encode(llm_response, convert_to_tensor=True)
    llm_entities = extract_entities(llm_response)
    similarities = {}
    entity_matches = {}
    missing_points = {}
    extra_points = {}

    for source, texts in reference_texts.items():
        max_similarity = 0
        matched_entities = set()
        for text in texts['cause']:
            ref_embedding = model.encode(text, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(llm_embedding, ref_embedding).item()
            if similarity > max_similarity:
                max_similarity = similarity
                matched_entities = extract_entities(text).intersection(llm_entities)
                missing_points[source] = extract_entities(text) - llm_entities
                extra_points[source] = llm_entities - extract_entities(text)
        similarities[source] = max_similarity
        entity_matches[source] = matched_entities

    return similarities, entity_matches, missing_points, extra_points

# Example usage
llm_response = llm_response_df.iloc[0]['Cause']
similarities, entity_matches, missing_points, extra_points = fact_check(llm_response, reference_texts)

print(f"LLM Response: {llm_response}")
print(f"Similarities: {similarities}")
print(f"Entity Matches: {entity_matches}")
print(f"Missing Points: {missing_points}")
print(f"Extra Points: {extra_points}")

# Save results to CSV file
results_df = pd.DataFrame({
    'Source': list(similarities.keys()),
    'Similarity': list(similarities.values()),
    'Entity Matches': [', '.join(entities) for entities in entity_matches.values()],
    'Missing Points': [', '.join(points) for points in missing_points.values()],
    'Extra Points': [', '.join(points) for points in extra_points.values()]
})

results_df.to_csv('fact_check_results.csv', index=False)

print("Results saved to fact_check_results.csv")
"""

In [ ]:
"""
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load LLM response
llm_response_df = pd.read_csv('/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/Llama_advice.csv')

# Load reference texts
reference_files = {
    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",
    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",
    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",
}

reference_texts = {}
for source, file_path in reference_files.items():
    reference_texts[source] = pd.read_csv(file_path)

# Initialize a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return set([ent.text for ent in doc.ents])

# Function to compare LLM response with reference texts and assign factual accuracy marks
def fact_check(llm_response, reference_texts):
    llm_embedding = model.encode(llm_response, convert_to_tensor=True)
    llm_entities = extract_entities(llm_response)
    similarities = {}
    entity_matches = {}
    missing_points = {}
    extra_points = {}
    factual_accuracy_marks = {}

    for source, texts in reference_texts.items():
        max_similarity = 0
        matched_entities = set()
        for text in texts['cause']:
            ref_embedding = model.encode(text, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(llm_embedding, ref_embedding).item()
            if similarity > max_similarity:
                max_similarity = similarity
                matched_entities = extract_entities(text).intersection(llm_entities)
                missing_points[source] = extract_entities(text) - llm_entities
                extra_points[source] = llm_entities - extract_entities(text)
        similarities[source] = max_similarity
        entity_matches[source] = matched_entities

        # Calculate factual accuracy marks
        marks = max_similarity * 100  # Base similarity score out of 100
        marks += len(matched_entities) * 5  # Add points for each matched entity
        marks -= len(missing_points[source]) * 5  # Deduct points for each missing entity
        marks -= len(extra_points[source]) * 5  # Deduct points for each extra entity
        factual_accuracy_marks[source] = marks

    return similarities, entity_matches, missing_points, extra_points, factual_accuracy_marks

# Example usage
llm_response = llm_response_df.iloc[0]['Cause']
similarities, entity_matches, missing_points, extra_points, factual_accuracy_marks = fact_check(llm_response, reference_texts)

print(f"LLM Response: {llm_response}")
print(f"Similarities: {similarities}")
print(f"Entity Matches: {entity_matches}")
print(f"Missing Points: {missing_points}")
print(f"Extra Points: {extra_points}")
print(f"Factual Accuracy Marks: {factual_accuracy_marks}")

# Save results to CSV file
results_df = pd.DataFrame({
    'Source': list(similarities.keys()),
    'Similarity': list(similarities.values()),
    'Entity Matches': [', '.join(entities) for entities in entity_matches.values()],
    'Missing Points': [', '.join(points) for points in missing_points.values()],
    'Hallucinated Points': [', '.join(points) for points in extra_points.values()],
    'Factual Accuracy Marks': list(factual_accuracy_marks.values())
})

results_df.to_csv('fact_check_results.csv', index=False)

print("Results saved to fact_check_results.csv")
"""

In [ ]:
"""
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load LLM response
llm_response_df = pd.read_csv('/content/drive/MyDrive/P2/LLM/LLL-Advice/Results/Llama_advice.csv')

# Load reference texts
reference_files = {
    "MayoClinic": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MayoClinic.csv",
    "Healthline": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/Healthline.csv",
    "MedlinePlus": "/content/drive/MyDrive/P2/LLM/LLL-Advice/sources/MedlinePlus.csv",
}

reference_texts = {}
for source, file_path in reference_files.items():
    reference_texts[source] = pd.read_csv(file_path)

# Initialize a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return set([ent.text for ent in doc.ents])

# Function to compare LLM response with reference texts and calculate factual accuracy
def fact_check(llm_response, reference_texts):
    llm_embedding = model.encode(llm_response, convert_to_tensor=True)
    llm_entities = extract_entities(llm_response)
    similarities = {}
    entity_matches = {}
    missing_points = {}
    extra_points = {}
    factual_accuracy = {}

    for source, texts in reference_texts.items():
        max_similarity = 0
        matched_entities = set()
        reference_entities = set()

        for text in texts['cause']:
            ref_embedding = model.encode(text, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(llm_embedding, ref_embedding).item()
            if similarity > max_similarity:
                max_similarity = similarity
                matched_entities = extract_entities(text).intersection(llm_entities)
                reference_entities.update(extract_entities(text))

        missing_points[source] = reference_entities - llm_entities
        extra_points[source] = llm_entities - reference_entities

        # Calculate factual accuracy
        total_reference_points = len(reference_entities)
        factual_accuracy[source] = ((total_reference_points - len(missing_points[source])) / total_reference_points) * 100 if total_reference_points > 0 else 0

        similarities[source] = max_similarity
        entity_matches[source] = matched_entities

    return similarities, entity_matches, missing_points, extra_points, factual_accuracy

# Example usage
llm_response = llm_response_df.iloc[0]['Cause']
similarities, entity_matches, missing_points, extra_points, factual_accuracy = fact_check(llm_response, reference_texts)

print(f"LLM Response: {llm_response}")
print(f"Similarities: {similarities}")
print(f"Entity Matches: {entity_matches}")
print(f"Missing Points: {missing_points}")
print(f"Extra Points: {extra_points}")
print(f"Factual Accuracy: {factual_accuracy}")

# Save results to CSV file
results_df = pd.DataFrame({
    'Source': list(similarities.keys()),
    'Similarity': list(similarities.values()),
    'Entity Matches': [', '.join(entities) for entities in entity_matches.values()],
    'Missing Points': [', '.join(points) for points in missing_points.values()],
    'Extra Points': [', '.join(points) for points in extra_points.values()],
    'Factual Accuracy (%)': list(factual_accuracy.values())
})

results_df.to_csv('fact_check_results.csv', index=False)

print("Results saved to fact_check_results.csv")
"""